In [28]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
f=open('data/application_train.csv')
app_train = pd.read_csv(f)
f2=open('data/application_test.csv')
app_test = pd.read_csv(f2)

In [29]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le_count = 0
for col in app_train:
    if app_train[col].dtype == 'object':
        # 如果只有兩種值的類別型欄位
        if len(list(app_train[col].unique())) <= 2:
            # 就做 Label Encoder
            le.fit(app_train[col])
            app_train[col] = le.transform(app_train[col])
            app_test[col] = le.transform(app_test[col])
            
            # 紀錄有多少個 columns 被標籤編碼過
            le_count += 1
            
# 標籤編碼 (2種類別) 欄位轉 One Hot Encoding            
app_train = pd.get_dummies(app_train)
app_test = pd.get_dummies(app_test)

In [30]:
app_train['DAYS_EMPLOYED_ANOM'] = app_train["DAYS_EMPLOYED"] == 365243
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)
app_test['DAYS_EMPLOYED_ANOM'] = app_test["DAYS_EMPLOYED"] == 365243
app_test["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace = True)
app_train['DAYS_BIRTH'] = abs(app_train['DAYS_BIRTH'])
app_test['DAYS_BIRTH'] = abs(app_test['DAYS_BIRTH'])

In [31]:
train_labels = app_train['TARGET']
app_train, app_test = app_train.align(app_test, join = 'inner', axis = 1)

In [13]:
app_test.shape

(48744, 240)

In [32]:
from sklearn.preprocessing import MinMaxScaler, Imputer
train = app_train
features = list(train.columns)
test = app_test.copy()

imputer = Imputer(strategy = 'median')
scaler = MinMaxScaler(feature_range = (0, 1))
imputer.fit(train)


Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)

In [96]:
import math
l=[]
for a in train["DAYS_EMPLOYED"]:
    if math.isnan(a):
        pass
    else:
        l.append(a)

len(train["DAYS_EMPLOYED"])
nanRecord=np.isnan(train["DAYS_EMPLOYED"])
len(l)
for i in range(len(nanRecord)):
    if bool(nanRecord[i])==True:
        #print (i)
        pass
b=nanRecord[0]


-637.0

In [98]:
sum0=0
for a in l:
    sum0+=a
sum0/len(l)

-2384.1693246132063

In [99]:
train = imputer.transform(train)

In [108]:
train
app_train["DAYS_EMPLOYED"]
for i in range(len(app_train.columns)):
    if app_train.columns[i]=="DAYS_EMPLOYED":
        print(i)


11


In [109]:
test = imputer.transform(app_test)

In [110]:
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)

In [20]:
print('Training data shape: ', train.shape)
print('Testing data shape: ', test.shape)

Training data shape:  (307511, 240)
Testing data shape:  (48744, 240)


In [111]:
from sklearn.linear_model import LogisticRegression

# 設定模型與模型參數
log_reg = LogisticRegression(C = 0.0001)

# 使用 Train 資料訓練模型
log_reg.fit(train, train_labels)

LogisticRegression(C=0.0001, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [113]:
log_reg_pred = log_reg.predict_proba(test)[:, 1]

In [114]:
submit = app_test[['SK_ID_CURR']]
submit['TARGET'] = log_reg_pred

submit.head()

/home/air/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,SK_ID_CURR,TARGET
0,100001,0.065051
1,100005,0.126401
2,100013,0.081239
3,100028,0.061509
4,100038,0.128308


In [115]:

submit.to_csv('20190511.csv',index=False)